## 5. Build Iot Edge Module - Docker Image
If not already, you should run first jupyter notebook (01_setup_environment.ipynb) in this sample to set the global variables.

### 5.1. Get global variables
We will read the previously stored variables

In [2]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

mlSolutionPath = get_key(envPath, "mlSolutionPath")
containerImageName = get_key(envPath, "containerImageName")
localContainerRegServiceName = get_key(envPath, "localContainerRegServiceName")
containerRegServiceName = get_key(envPath, "containerRegServiceName")

In [5]:
print(containerImageName)
print(localContainerRegServiceName)

mhlva01aimodule
mhregistry:55000


In [2]:
# remove possible python cache from previous local tests
!rm -rf $mlSolutionPath/__pycache__

### 5.2 Create Web Application & Server for our ml solution

In [ ]:
%%writefile $mlSolutionPath/app.py

from flask import Flask, request, Response
from PIL import Image
import logging
import json
import io
from score import AnalyticsAPI

app = Flask(__name__)
analyticsAPI = AnalyticsAPI(workDir=".")

@app.route("/score", methods = ['POST'])
def scoreRRS():
    global analyticsAPI
    #analyticsAPI.logger.info("[AI EXT] Received scoring request. Header: {0}".format(json.dumps(dict(request.headers))))

    try:    
        if request.headers['Content-Type'] != 'image/jpeg':
            analyticsAPI.logger.info("[AI EXT] Non JPEG content sent. Exiting the scoring event...")
            return Response(json.dumps({}), status= 415, mimetype ='application/json')

        # get request as byte stream
        reqBody = request.get_data(False)

        # convert from byte stream
        inMemFile = io.BytesIO(reqBody)

        # load a sample image
        pilImage = Image.open(inMemFile)

        # call scoring function
        result = analyticsAPI.score(pilImage)            

        analyticsAPI.logger.info("[AI EXT] Sending response.")
        return Response(result, status= 200, mimetype ='application/json')

    except Exception as e:
        analyticsAPI.logger.info("[AI EXT] Exception (scoreRRS): {0}".format(str(e)))
        return Response(json.dumps({}), status= 200, mimetype ='application/json')   
    
@app.route("/")
def healthy():
    return "Healthy"

# Version
@app.route('/version', methods = ['GET'])
def version_request():
    global analyticsAPI
    return analyticsAPI.version()

# About
@app.route('/about', methods = ['GET'])
def about_request():
    global analyticsAPI
    return analyticsAPI.about()

if __name__ == "__main__":
    while not analyticsAPI.initialized:
        logger.info("[AI EXT] Waiting AI module to be initialized. (app.py)")
        time.sleep(1)
        
    app.run(host='0.0.0.0', port=5444)

No need to make any change here but 5444 is the internal port of the webserver app that listens the requests. Later we will map it to different port to expose to external (see next cell)

In [ ]:
%%writefile $mlSolutionPath/wsgi.py
from app import app as application

def create():
    print("[AI EXT] Initialising lva ai extension web app")
    application.run(host='127.0.0.1', port=5444)

In [ ]:
import os
os.makedirs(os.path.join(mlSolutionPath, "nginx"), exist_ok=True)

Exposed port of the web app is now 5001 (while the internal one is 5444)

In [ ]:
%%writefile $mlSolutionPath/nginx/app
server {
    listen 5001;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5444;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

In [ ]:
%%writefile $mlSolutionPath/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

In [ ]:
%%writefile $mlSolutionPath/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('[AI EXT] READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('[AI EXT] Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('[AI EXT] Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('[AI EXT] RESULT 2\nOK')

main()

In [ ]:
import os
os.makedirs(os.path.join(mlSolutionPath, "etc"), exist_ok=True)

In [ ]:
%%writefile $mlSolutionPath/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

In [5]:
%%writefile $mlSolutionPath/requirements.txt
pillow<7.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3
six==1.13.0
opencv-python==4.1.1.26
imgaug==0.3.0
torch==1.2
cython==0.29.14
addict==2.2.1
azure-storage-blob==12.3.0
python-dotenv==0.13.0

Overwriting lva_ai_solution/requirements.txt


### 5.3. Docker File to containerize ml solution and web app server

Instead of installing latest ONNX Runtime python package, we will build a base docker image with ONNX Runtime to be compiled from latest source code.

In [ ]:
# %%writefile $mlSolutionPath/install_onnxruntime.sh
# #!/bin/bash
# DEBIAN_FRONTEND=noninteractive
# apt-get update && apt-get install -y --no-install-recommends \
#         git \
#         wget \
#         zip \
#         ca-certificates \
#         build-essential \
#         curl \
#         libcurl4-openssl-dev \
#         libssl-dev \
#         python3-dev \
#         sudo \
#         nano

# cd ${WORK_DIR}

# # Dependencies: conda
# wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ${WORK_DIR}/miniconda.sh --no-check-certificate
# /bin/bash ${WORK_DIR}/miniconda.sh -b -p /opt/miniconda
# /opt/miniconda/bin/conda clean -ya

# pip install numpy
# rm -rf /opt/miniconda/pkgs

# # Dependencies: cmake
# wget --quiet https://github.com/Kitware/CMake/releases/download/v3.14.3/cmake-3.14.3-Linux-x86_64.tar.gz
# tar zxf cmake-3.14.3-Linux-x86_64.tar.gz

# # clone latest ONNX runtime source for compile
# git clone --single-branch --branch ${ONNXRUNTIME_SERVER_BRANCH} --recursive ${ONNXRUNTIME_REPO} onnxruntime

# cd ${WORK_DIR}/onnxruntime
# /bin/bash  ./build.sh --config Release --build_wheel --update --build --parallel --cmake_extra_defines ONNXRUNTIME_VERSION=$(cat ./VERSION_NUMBER)

# pip install ${WORK_DIR}/onnxruntime/build/Linux/Release/dist/*.whl

In [6]:
%%writefile $mlSolutionPath/install_mlsolution.sh
#!/bin/bash
DEBIAN_FRONTEND=noninteractive

apt-get update && apt-get install -y --no-install-recommends \
        nginx \
        supervisor

rm -rf /var/lib/apt/lists/*

rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -ya


Overwriting lva_ai_solution/install_mlsolution.sh


In [ ]:
# %%writefile $mlSolutionPath/Dockerfile
# FROM ubuntu:18.04
# MAINTAINER Micheleen Harris

# ARG ONNXRUNTIME_REPO=https://github.com/Microsoft/onnxruntime
# ARG ONNXRUNTIME_SERVER_BRANCH=master
# ARG WORK_DIR=/code

# RUN mkdir /code

# WORKDIR ${WORK_DIR}
# ENV PATH /opt/miniconda/bin:${WORK_DIR}/cmake-3.14.3-Linux-x86_64/bin:${PATH}
# ENV WORK_DIR ${WORK_DIR}

# RUN apt-get update &&\
#     apt-get install -y bash

# # Prepare onnxruntime repository & build onnxruntime
# COPY install_onnxruntime.sh ${WORK_DIR}

# RUN cd ${WORK_DIR} && \
#     /bin/bash ${WORK_DIR}/install_onnxruntime.sh && \
#     rm -rf *

# ADD . /code/
# ADD etc /etc

# # Setup ml solution
# COPY install_mlsolution.sh ${WORK_DIR}

# RUN cd ${WORK_DIR} && \
#     /bin/bash ${WORK_DIR}/install_mlsolution.sh

# EXPOSE 5001
# CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

In [4]:
%%writefile $mlSolutionPath/Dockerfile
FROM nvidia/cuda:10.1-cudnn7-devel-ubuntu18.04
MAINTAINER Micheleen Harris

ARG WORK_DIR=/code

ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
ENV PATH /opt/conda/bin:$PATH
ENV TORCH_CUDA_ARCH_LIST=Volta;Turing;Kepler+Tesla

RUN mkdir /code

RUN apt-get update && apt-get install -y --no-install-recommends \
         build-essential \
         cmake \
         git \
         curl \
         wget \
         ca-certificates \
         libjpeg-dev \
         libpng-dev \
         swig \
         libgtk2.0-dev && \
     rm -rf /var/lib/apt/lists/*

WORKDIR ${WORK_DIR}

RUN wget --quiet https://repo.continuum.io/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ${WORK_DIR}/miniconda.sh && \
    /bin/bash ${WORK_DIR}/miniconda.sh -b -p /opt/conda && \
    rm ${WORK_DIR}/miniconda.sh && \
    ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh && \
    echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && \
    echo "conda activate base" >> ~/.bashrc

# This must be done before pip so that requirements.txt is available
WORKDIR ${WORK_DIR}
COPY . .

# Install python requirements for project
RUN CMAKE_PREFIX_PATH="$(dirname $(which conda))/../" \
    pip install -r requirements.txt

# Compile ops
RUN chmod +x compile_ops.sh && \
    CMAKE_PREFIX_PATH="$(dirname $(which conda))/../" \
    ./compile_ops.sh

RUN chmod -R a+w .

ADD . /code/
ADD etc /etc

# Setup ml solution
COPY install_mlsolution.sh ${WORK_DIR}

RUN cd ${WORK_DIR} && \
    /bin/bash ${WORK_DIR}/install_mlsolution.sh

EXPOSE 5001
CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Overwriting lva_ai_solution/Dockerfile


### 5.4. Create Local Docker Image
We create the image locally, without hostring it yet in a container registry like docker.com or ACR or local registry

Dont forget that you need to have pre-requisities mentioned in section 1... We are running docker command without "sudo"

In [3]:
%%bash -s "$containerImageName" "$mlSolutionPath"
docker build -t $1 --file ./$2/Dockerfile ./$2


Step 1/20 : FROM nvidia/cuda:10.1-cudnn7-devel-ubuntu18.04
10.1-cudnn7-devel-ubuntu18.04: Pulling from nvidia/cuda
7ddbc47eeb70: Already exists
c1bbdc448b72: Already exists
8c3b70e39044: Already exists
45d437916d57: Already exists
d8f1569ddae6: Already exists
85386706b020: Pulling fs layer
ee9b457b77d0: Pulling fs layer
be4f3343ecd3: Pulling fs layer
30b4effda4fd: Pulling fs layer
b398e882f414: Pulling fs layer
b398e882f414: Waiting
30b4effda4fd: Waiting
ee9b457b77d0: Download complete
85386706b020: Verifying Checksum
85386706b020: Download complete
85386706b020: Pull complete
ee9b457b77d0: Pull complete
b398e882f414: Verifying Checksum
b398e882f414: Download complete
30b4effda4fd: Verifying Checksum
30b4effda4fd: Download complete
be4f3343ecd3: Download complete
be4f3343ecd3: Pull complete
30b4effda4fd: Pull complete
b398e882f414: Pull complete
Digest: sha256:557de4ba2cb674029ffb602bed8f748d44d59bb7db9daa746ea72a102406d3ec
Status: Downloaded newer image for nvidia/cuda:10.1-cudnn7-de

The command '/bin/sh -c cd ${WORK_DIR} &&     /bin/bash ${WORK_DIR}/install_mlsolution.sh' returned a non-zero code: 127


CalledProcessError: Command 'b'docker build -t $1 --file ./$2/Dockerfile ./$2\n'' returned non-zero exit status 127.

### 5.5. Host the local docker image in local docker registry
We will be hosting the docker image on a local registry which is either on the edge device, or on the development device which is on the same network as the edge device. We prefer this method only at the development stage as:
- Docker image size ~ 800 Mb
- Hosting image on Azure Container Registry (ACR) means each time we compile the image, we will send it, push it to cloud and then pull all the way back to edge device. In case of low Internet connection bandwidth, this may be an issue or time consuming operation.
- There is an option where you can send smaller size DockerFile and dependencies to ACR and compile there. But in this case, 1) again you need to pull back the 1-2 GB size image to edge device 2) Even in case of a small code change in the last layers of the docker image, ACR will re-compile every layer from scratc which may take additional ~40 minutes...

So for above reasons, at the development and testing stages, we will use local container registry. Below is very simple steps to create your own docker registry by running the following bash commands manually:  

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
In the below instructions we use **"mkregistry:55000"** as static text which is stored in the variable **localContainerRegServiceName**. Value of this variable set in the first step/section of this sample. You can change this value to anything else but in case, you should update the commands in this static text with the new value.


1) Create a folder to host local docker registry configuration folder (i.e. we name it "mkregistry").
```shell
mkdir mkregistry
```
2) Inside the "mkregistry" directory, create a congiguration file named "docker-compose.yml" with the following content in:

```yml
version: '3.0'
 
services:
  mkregistry:
    image: registry:latest
    container_name: mkregistry
    volumes:
      - registry:/var/lib/registry
    ports:
      - "55000:5000"
    restart: unless-stopped
volumes:
  registry:
```

3) Install "docker-compose" tool. Run the following shell command to install it.  
```shell
sudo apt-get -y install docker-compose
```

4) Run the following command inside the "mkregistry" directory with "docker-compose.yml" file in it.  
```shell
sudo docker-compose up -d
```

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
> When you restart the machine etc. you need to re-run above command to start the local regisrty service (you can search for methods to run it as auto run service)
when you type "docker container ls" shell command. You should see the following line to confirm that the service is running, you can validate with below next step.
```
f63fd9492a03        registry:latest                              "/entrypoint.sh /etc…"    20 hours ago        Up 19 hours         0.0.0.0:55000->5000/tcp                                                mkregistry
m
```

5) In your Internet explorer - web browser, you browse to following address to see the images in your local repository (initially it is empty...)  
```
http://localhost:55000/v2/_catalog
```

6) Edit "hosts" file to assign a name resolution setting for local docker registry. First open the "/etc/hosts" file with your favorite text editor:  
```shell
sudo nano /etc/hosts
```

than add the following line under "127.0.0.0 localhost"  

```shell
127.0.0.1	mkregistry
```

so the content of "/etc/hosts" file should look like something:  
```shell
127.0.0.1	localhost
127.0.0.1	mkregistry
127.0.1.1	mknuc01

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
```

thats all, now we have local docker registy with following addres: mkregistry:55000  

With following code, we will tag our local image and pull it into our local docker registry. So when we deploy our IotEdge manifest to IoT Hub, the edge devices will pull it from this registry. In production stage, we will host the image in ACR, more centric and managable store.

In [ ]:
%%bash -s "$containerImageName" "$localContainerRegServiceName"
docker tag $1 $2/$1
docker push $2/$1 

In [9]:
print(containerImageName)
print(localContainerRegServiceName)

mhlva01aimodule
mhregistry:55000


<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
If you are in production, you can push the image into ACR (and not local repository). Uncomment and use the following command to push it into ACR. Also dont forget to uncomment another line in the next step/section of the sample where you deploy the image into edge devices. So the edge device will pull modules from the ACR and not from local registry...

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Warning] </span>  
Execution of following cell may take up to ~40 - 45 minutes to complete! It will be using Azure Constainer Registry (ACR) to compile a docker image.

In [ ]:
#!az acr build --image $containerImageName --registry $containerRegServiceName --file ./$mlSolutionPath/Dockerfile ./$mlSolutionPath